In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
base_model = tf.keras.applications.MobileNet(input_shape=(128, 128, 3), 
                                               include_top=False, weights='imagenet')

base_model.trainable = False



17225924/17225924 [==============================] - 1s 0us/step


In [3]:
from tensorflow.keras.regularizers import l2

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Regularizacija
    layers.Dropout(0.6),  # Smanjuje overfitting
    layers.Dense(20, activation='softmax')  # 20 je broj klasa
])

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss= 'categorical_crossentropy', # 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    'Train',  # Putanja do foldera sa podacima za treniranje
    target_size=(128, 128),     # Resize na 96x96
    batch_size=16,
    class_mode= 'categorical' ) # 'sparse')

Found 949 images belonging to 20 classes.


In [7]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    'Validate',  # Putanja do foldera sa podacima za treniranje
    target_size=(128, 128),     # Resize na 96x96
    batch_size=16,
    class_mode= 'categorical') #'sparse')

Found 119 images belonging to 20 classes.


In [8]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

model.summary()

Epoch 1/50


60/60 [==============================] - 12s 166ms/step - loss: 4.6187 - accuracy: 0.0664 - val_loss: 3.3699 - val_accuracy: 0.0672 - lr: 1.0000e-04
Epoch 2/50
60/60 [==============================] - 8s 139ms/step - loss: 3.6193 - accuracy: 0.1138 - val_loss: 3.0733 - val_accuracy: 0.1681 - lr: 1.0000e-04
Epoch 3/50
60/60 [==============================] - 9s 149ms/step - loss: 3.1302 - accuracy: 0.1760 - val_loss: 2.8556 - val_accuracy: 0.2941 - lr: 1.0000e-04
Epoch 4/50
60/60 [==============================] - 9s 150ms/step - loss: 2.9128 - accuracy: 0.2234 - val_loss: 2.6824 - val_accuracy: 0.3361 - lr: 1.0000e-04
Epoch 5/50
60/60 [==============================] - 10s 164ms/step - loss: 2.6701 - accuracy: 0.2940 - val_loss: 2.5007 - val_accuracy: 0.4286 - lr: 1.0000e-04
Epoch 6/50
60/60 [==============================] - 10s 155ms/step - loss: 2.4636 - accuracy: 0.3751 - val_loss: 2.3375 - val_accuracy: 0.4622 - lr: 1.0000e-04
Epoch 7/50
60/60 [=======================

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('mobilenet_bird_classifier_v2.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Emma\AppData\Local\Temp\tmpakjksl26\assets


INFO:tensorflow:Assets written to: C:\Users\Emma\AppData\Local\Temp\tmpakjksl26\assets


In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalizacija slika

test_generator = test_datagen.flow_from_directory(
    'Test',  # Putanja do foldera sa test podacima
    target_size=(128, 128),  # Resize na 96x96
    batch_size=16,
    class_mode= 'categorical') # 'sparse')

Found 120 images belonging to 20 classes.


In [11]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

# Ispis rezultata
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

8/8 [==============================] - 1s 150ms/step - loss: 1.1554 - accuracy: 0.7667
Test Loss: 1.1554204225540161
Test Accuracy: 0.7666666507720947


In [12]:
interpreter = tf.lite.Interpreter(model_path='mobilenet_bird_classifier.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'serving_default_mobilenet_1.00_224_input:0', 'index': 0, 'shape': array([  1, 124, 124,   3]), 'shape_signature': array([ -1, 124, 124,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 95, 'shape': array([ 1, 20]), 'shape_signature': array([-1, 20]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [13]:
import numpy as np

with open("mobilenet_bird_classifier_v2.tflite", "rb") as f:
    model_bytes = f.read()

model_array = np.frombuffer(model_bytes, dtype=np.uint8)

with open("model_v2.h", "w") as f:
    f.write("#ifndef MODEL_H\n#define MODEL_H\n\n")
    f.write("const unsigned char model_tflite[] = {")
    
    for i, byte in enumerate(model_array):
        if i % 12 == 0:
            f.write("\n    ")
        f.write(f"0x{byte:02x}, ")

    f.write("\n};\n")
    f.write(f"const unsigned int model_tflite_len = {len(model_array)};\n\n")
    f.write("#endif // MODEL_H\n")
